In [ ]:
!pip install requests

In [ ]:
import requests
import json
import getpass

# 1. Login to NeuroHub through the API

In [ ]:
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json',
}
data = {
    'login': input('username'),
    'password': getpass.getpass('password')
}
response = requests.post('https://portal.cbrain.mcgill.ca/session', headers=headers, data=data)
if response.status_code == 200:
    print("Login success")
    print(response.content)
    jsonResponse = response.json()
    token = jsonResponse["cbrain_api_token"] #Token identifies your user session for you to use the API.
else:
    print("Login failure")

# 2. Downloading NeuroHub files through the API

##### -Given file IDs.

In [ ]:
fileID = str(3319815)

headers = {
    'Accept': 'application/json',
}
params = (
    ('id', fileID),
    ('cbrain_api_token', token),
)
url = 'https://portal.cbrain.mcgill.ca//userfiles/' + fileID + '/content'
response = requests.get(url, headers=headers, params=params, allow_redirects=True)
if response.status_code == 200:
    open('sub-1004359_ses-PREBL00_run-001_T1w.nii.gz', 'wb').write(response.content)
    print("Downloaded file")
else:
    print('File download failure')

In [ ]:
fileID = str(3319812)

headers = {
    'Accept': 'application/json',
}
params = (
    ('id', fileID),
    ('cbrain_api_token', token),
)
url = 'https://portal.cbrain.mcgill.ca//userfiles/' + fileID + '/content'
response = requests.get(url, headers=headers, params=params, allow_redirects=True)
if response.status_code == 200:
    open('sub-1004359_ses-PREBL00_task-encoding_run-001_bold.nii.gz', 'wb').write(response.content)
    print("Downloaded file")
else:
    print('File download failure')

# 3. Working with the downloaded files

##### Can use the downloaded files with popular libraries such as nibabel and nilearn
##### Adapted from the nilearn examples at https://nilearn.github.io/auto_examples/index.html

In [ ]:
import sys
#install the packages we are using
!{sys.executable} -m pip install nibabel
!{sys.executable} -m pip install nilearn
!{sys.executable} -m pip install matplotlib

In [ ]:
import nibabel as nib #loads the brain scan files
from nilearn import plotting #For visualization

In [ ]:
file_to_load = 'sub-1004359_ses-PREBL00_run-001_T1w.nii.gz' #specify filename as string
anat_file = nib.load(file_to_load) #loading an anatomical (T1 weighted) scan
print(anat_file.shape) #3D shape

file_to_load2 = 'sub-1004359_ses-PREBL00_task-encoding_run-001_bold.nii.gz'
bold_file = nib.load(file_to_load2) #loading an fMRI (BOLD) scan
print(bold_file.shape)#4D shape

plotting.plot_img(anat_file) #General purpose plotting, no specific preset
plotting.plot_anat(anat_file) #Anatomical image plotting

# 4. Smoothing

In [ ]:
from nilearn import image
smooth_anat_img = image.smooth_img(anat_file, fwhm=3) #Apply a Gaussian filter along the three first dimensions of scan.
#fwhm parameter is the smoothing strength in mm
plotting.plot_img(smooth_anat_img) #General plotting function, compare to above.

# 5. fMRI visualization

In [ ]:
from nilearn.image.image import mean_img

mean_fmri = mean_img(bold_file) #Compute the mean of the images over time or the 4th dimension

from nilearn.plotting import plot_epi, show
plot_epi(mean_fmri) #plot cuts of the image (by default 3 cuts: Frontal, Axial, and Lateral)

# 6. Extract a brain mask

In [ ]:
from nilearn.masking import compute_epi_mask
mask_img = compute_epi_mask(bold_file) #Compute a brain mask from fMRI data, 3D volume representing where object is within 3D space

# Visualize it as an ROI
from nilearn.plotting import plot_roi
plot_roi(mask_img, mean_fmri) #Plot cuts of an ROI/mask image (by default 3 cuts: Frontal, Axial, and Lateral)

# 7. Applying the mask to extract the corresponding time series

In [ ]:
from nilearn.masking import apply_mask
masked_data = apply_mask(bold_file, mask_img) #Read the time series (extract signal) from the given Niimg-like object, using the mask.
#masked_data is in the form (timepoints, voxels).

import matplotlib.pyplot as plt
plt.figure(figsize=(7, 5))
plt.plot(masked_data[:150, :2]) #Plot first 150 timepoints from 2 voxels
plt.xlabel('Time [TRs]', fontsize=16)
plt.ylabel('Intensity', fontsize=16)
plt.xlim(0, 150)
plt.subplots_adjust(bottom=.12, top=.95, right=.95, left=.12)

show()

# 8. Independent component analysis (ICA) of fMRI
##### -Signal processing method to separate independent sources linearly mixed in several sensors
##### -Special case of blind source separation
##### -ICA will tell us to what degree each component contributes to the observed signal.

In [ ]:
from nilearn.input_data import NiftiMasker

# Generate 3D mask to remove background
masker = NiftiMasker(smoothing_fwhm=8, memory='nilearn_cache', memory_level=1,
                     mask_strategy='epi', standardize=True)
data_masked = masker.fit_transform(bold_file)

In [ ]:
from sklearn.decomposition import FastICA
n_components = 10
ica = FastICA(n_components=n_components, random_state=42)
components_masked = ica.fit_transform(data_masked.T).T

# Normalize estimated components, for thresholding to make sense
components_masked -= components_masked.mean(axis=0)
components_masked /= components_masked.std(axis=0)
# Threshold
import numpy as np
components_masked[np.abs(components_masked) < .8] = 0

# Invert the masking operation, to go back to a full 3D representation
component_img = masker.inverse_transform(components_masked)

In [ ]:
from nilearn import image
from nilearn.plotting import plot_stat_map, show

# Use the mean as a background
mean_img = image.mean_img(bold_file)

# Show some interesting components
plot_stat_map(image.index_img(component_img, 0), mean_img)
plot_stat_map(image.index_img(component_img, 1), mean_img)

show()